In [1]:
import io
import requests

import pandas as pd
import polars as pl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')

In [2]:
pdf = pd.read_parquet('../data/raw/ftransfer_elasticc_2023-02-15_946675/classId=121')

In [3]:
def extract_history(history_list: list, field: str) -> list:
    """Extract the historical measurements contained in the alerts
    for the parameter `field`.

    Parameters
    ----------
    history_list: list of dict
        List of dictionary from alert[history].
    field: str
        The field name for which you want to extract the data. It must be
        a key of elements of history_list
    
    Returns
    ----------
    measurement: list
        List of all the `field` measurements contained in the alerts.
    """
    if history_list is None:
        return []
    try:
        measurement = [obs[field] for obs in history_list]
    except KeyError:
        print('{} not in history data'.format(field))
        measurement = []

    return measurement

def extract_field(alert: dict, category: str, field: str, key: str) -> np.array:
    """ Concatenate current and historical observation data for a given field.
    
    Parameters
    ----------
    alert: dict
        Dictionnary containing alert data
    category: str
        prvDiaSources or prvDiaForcedSources
    field: str
        Name of the field to extract.
    
    Returns
    ----------
    data: np.array
        List containing previous measurements and current measurement at the
        end. If `field` is not in the category, data will be
        [alert['diaSource'][field]].
    """
    data = np.concatenate(
        [
            [alert[key][field]],
            extract_history(alert[category], field)
        ]
    )
    return data

In [4]:
pdf['target'] = 121

In [5]:
pdf["diaObject"][0]

{'decl': -49.696072651146864,
 'diaObjectId': 146402526,
 'hostgal2_dec': -999.0,
 'hostgal2_ellipticity': -9999.0,
 'hostgal2_mag_Y': 999.0,
 'hostgal2_mag_g': 999.0,
 'hostgal2_mag_i': 999.0,
 'hostgal2_mag_r': 999.0,
 'hostgal2_mag_u': 999.0,
 'hostgal2_mag_z': 999.0,
 'hostgal2_magerr_Y': 999.0,
 'hostgal2_magerr_g': 999.0,
 'hostgal2_magerr_i': 999.0,
 'hostgal2_magerr_r': 999.0,
 'hostgal2_magerr_u': 999.0,
 'hostgal2_magerr_z': 999.0,
 'hostgal2_ra': -999.0,
 'hostgal2_snsep': -9.0,
 'hostgal2_sqradius': -9999.0,
 'hostgal2_zphot': -999.0,
 'hostgal2_zphot_err': -999.0,
 'hostgal2_zphot_p50': None,
 'hostgal2_zphot_q000': -9.0,
 'hostgal2_zphot_q010': -9.0,
 'hostgal2_zphot_q020': -9.0,
 'hostgal2_zphot_q030': -9.0,
 'hostgal2_zphot_q040': -9.0,
 'hostgal2_zphot_q050': -9.0,
 'hostgal2_zphot_q060': -9.0,
 'hostgal2_zphot_q070': -9.0,
 'hostgal2_zphot_q080': -9.0,
 'hostgal2_zphot_q090': -9.0,
 'hostgal2_zphot_q100': -9.0,
 'hostgal2_zspec': -999.0,
 'hostgal2_zspec_err': -999.0,

In [6]:
print(pdf.columns)

Index(['alertId', 'diaSource', 'prvDiaSources', 'prvDiaForcedSources',
       'prvDiaNondetectionLimits', 'diaObject', 'cutoutDifference',
       'cutoutTemplate', 'brokerIngestTimestamp', 'timestamp',
       'rf_snia_vs_nonia', 'snn_snia_vs_nonia', 'snn_broad_class',
       'snn_broad_max_prob', 'cats_broad_class', 'cats_broad_max_prob',
       'cats_fine_class', 'cats_fine_max_prob', 'rf_agn_vs_nonagn',
       't2_broad_class', 't2_broad_max_prob', 'fink_broker_version',
       'fink_science_version', 'publisher', 'SNID', 'LIBID',
       'SIM_SEARCHEFF_MASK', 'GENTYPE', 'SIM_TEMPLATE_INDEX', 'ZCMB', 'ZHELIO',
       'ZCMB_SMEAR', 'RA', 'DEC', 'MWEBV', 'GALNMATCH', 'GALID', 'GALZPHOT',
       'GALZPHOTERR', 'GALSNSEP', 'GALSNDDLR', 'RV', 'AV', 'MU', 'LENSDMU',
       'PEAKMJD', 'MJD_DETECT_FIRST', 'MJD_DETECT_LAST', 'DTSEASON_PEAK',
       'PEAKMAG_u', 'PEAKMAG_g', 'PEAKMAG_r', 'PEAKMAG_i', 'PEAKMAG_z',
       'PEAKMAG_Y', 'SNRMAX', 'SNRMAX2', 'SNRMAX3', 'NOBS', 'NOBS_SATURATE',
     

In [7]:
pdf["NOBS"][0]

149

In [8]:
pdf['cpsFlux'] = pdf[['diaSource', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'psFlux', 'diaSource'), axis=1)
pdf['cpsFluxErr'] = pdf[['diaSource', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'psFluxErr', 'diaSource'), axis=1)
pdf['cfilterName'] = pdf[['diaSource', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'filterName', 'diaSource'), axis=1)
pdf['cmidPointTai'] = pdf[['diaSource', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'midPointTai', 'diaSource'), axis=1)

pdf['cZ'] = pdf[['diaObject', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'z_final', 'diaObject'), axis=1)
pdf['cZerr'] = pdf[['diaObject', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'z_final_err', 'diaObject'), axis=1)
pdf['cMwebv'] = pdf[['diaObject', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'mwebv', 'diaObject'), axis=1)
pdf['cRa'] = pdf[['diaObject', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'ra', 'diaObject'), axis=1)
pdf['cDecl'] = pdf[['diaObject', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'decl', 'diaObject'), axis=1)
pdf['cHostgal_ra'] = pdf[['diaObject', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'hostgal_ra', 'diaObject'), axis=1)
pdf['cHostgal_dec'] = pdf[['diaObject', 'prvDiaForcedSources']]\
    .apply(lambda x: extract_field(x, 'prvDiaForcedSources', 'hostgal_dec', 'diaObject'), axis=1)

cols = ['alertId', 'target', 'cmidPointTai', 'cpsFlux', 'cpsFluxErr', 'cfilterName', 'cZ', \
        'cZerr', 'cMwebv', 'cRa', 'cDecl', 'cHostgal_ra', 'cHostgal_dec', \
        'SNID', 'NOBS']
sub = pdf[cols]
sub

z_final not in history data
z_final not in history data
z_final not in history data
z_final not in history data
z_final not in history data
z_final_err not in history data
z_final_err not in history data
z_final_err not in history data
z_final_err not in history data
z_final_err not in history data
mwebv not in history data
mwebv not in history data
mwebv not in history data
mwebv not in history data
mwebv not in history data
ra not in history data
ra not in history data
ra not in history data
ra not in history data
ra not in history data
decl not in history data
decl not in history data
decl not in history data
decl not in history data
decl not in history data
hostgal_ra not in history data
hostgal_ra not in history data
hostgal_ra not in history data
hostgal_ra not in history data
hostgal_ra not in history data
hostgal_dec not in history data
hostgal_dec not in history data
hostgal_dec not in history data
hostgal_dec not in history data
hostgal_dec not in history data


,alertId,target,cmidPointTai,cpsFlux,cpsFluxErr,cfilterName,cZ,cZerr,cMwebv,cRa,cDecl,cHostgal_ra,cHostgal_dec,SNID,NOBS
0,292805052048,121,[60607.0736],[517.75732421875],[221.70233154296875],[i],[0.16344062983989716],[0.16808000206947327],[0.009014381095767021],[0.9676724344437256],[-49.696072651146864],[0.9677252769470215],[-49.69614791870117],146402526,149
1,236811186028,121,[60320.1313],[1169.298583984375],[196.1234130859375],[i],[0.1338547170162201],[0.1252799928188324],[0.00950966402888298],[69.52126616511217],[-50.38369680355572],[69.52117156982422],[-50.38372802734375],118405593,127
2,181886744009,121,[60549.1873],[1041.65673828125],[273.07745361328125],[r],[0.23332324624061584],[0.22684000432491302],[0.006137879565358162],[2.553227017279791],[-43.31437494725598],[2.5532901287078857],[-43.3144645690918],90943372,237
3,221478330013,121,[60609.3722],[2575.27392578125],[692.6956787109375],[r],[0.050953783094882965],[0.0010000000474974513],[0.014673247002065182],[57.33047750559256],[-53.56728248804294],[57.33053207397461],[-53.567298889160156],110739165,115
4,250750812072,121,[60466.004],[1167.426513671875],[206.24415588378906],[i],[0.1325898915529251],[0.0010000000474974513],[0.08095170557498932],[188.22928460733328],[-40.88442885410509],[188.22918701171875],[-40.88450622558594],125375406,157
5,164227482030,121,"[60340.3594, 60314.2508, 60314.2756, 60314.286...","[4735.275390625, -231.21820068359375, -552.134...","[644.0177001953125, 310.8291015625, 462.648345...","[z, i, z, z, Y, r, i, r, r, r, r, r, i, i, Y, ...",[0.11739850789308548],[0.13095000386238098],[0.05223875492811203],[157.65726394121165],[-22.807671218881783],[157.65725708007812],[-22.807472229003906],82113741,163
6,164227482031,121,"[60340.3716, 60314.2508, 60314.2756, 60314.286...","[4800.42529296875, -231.21820068359375, -552.1...","[693.317138671875, 310.8291015625, 462.6483459...","[z, i, z, z, Y, r, i, r, r, r, r, r, i, i, Y, ...",[0.11739850789308548],[0.13095000386238098],[0.05223875492811203],[157.65726394121165],[-22.807671218881783],[157.65725708007812],[-22.807472229003906],82113741,163
7,164227482029,121,"[60340.3479, 60314.2508, 60314.2756, 60314.286...","[4983.572265625, -231.21820068359375, -552.134...","[417.4020080566406, 310.8291015625, 462.648345...","[i, i, z, z, Y, r, i, r, r, r, r, r, i, i, Y, ...",[0.11739850789308548],[0.13095000386238098],[0.05223875492811203],[157.65726394121165],[-22.807671218881783],[157.65725708007812],[-22.807472229003906],82113741,163
8,38666284032,121,[60593.2405],[791.9251098632812],[153.12400817871094],[g],[0.16801081597805023],[0.1820099949836731],[0.012775149196386337],[24.38803778053321],[-45.10943347471609],[24.38807487487793],[-45.10942840576172],19333142,236
9,38666284033,121,[60593.2654],[1764.031982421875],[156.42236328125],[r],[0.16801081597805023],[0.1820099949836731],[0.012775149196386337],[24.38803778053321],[-45.10943347471609],[24.38807487487793],[-45.10942840576172],19333142,236


In [9]:
sub.shape

(40, 15)

In [10]:
def f(x):
    y = len(list(set(x))) > 1
    #sample function
    return y 

df = sub["cfilterName"].apply(f)
sub = sub[df].reset_index()

In [11]:
def f(x):
    y = len(x) > 1
    #sample function
    return y 

df = sub["cmidPointTai"].apply(f)
sub = sub[df].reset_index(drop=True)

In [12]:
sub

,index,alertId,target,cmidPointTai,cpsFlux,cpsFluxErr,cfilterName,cZ,cZerr,cMwebv,cRa,cDecl,cHostgal_ra,cHostgal_dec,SNID,NOBS
0,5,164227482030,121,"[60340.3594, 60314.2508, 60314.2756, 60314.286...","[4735.275390625, -231.21820068359375, -552.134...","[644.0177001953125, 310.8291015625, 462.648345...","[z, i, z, z, Y, r, i, r, r, r, r, r, i, i, Y, ...",[0.11739850789308548],[0.13095000386238098],[0.05223875492811203],[157.65726394121165],[-22.807671218881783],[157.65725708007812],[-22.807472229003906],82113741,163
1,6,164227482031,121,"[60340.3716, 60314.2508, 60314.2756, 60314.286...","[4800.42529296875, -231.21820068359375, -552.1...","[693.317138671875, 310.8291015625, 462.6483459...","[z, i, z, z, Y, r, i, r, r, r, r, r, i, i, Y, ...",[0.11739850789308548],[0.13095000386238098],[0.05223875492811203],[157.65726394121165],[-22.807671218881783],[157.65725708007812],[-22.807472229003906],82113741,163
2,7,164227482029,121,"[60340.3479, 60314.2508, 60314.2756, 60314.286...","[4983.572265625, -231.21820068359375, -552.134...","[417.4020080566406, 310.8291015625, 462.648345...","[i, i, z, z, Y, r, i, r, r, r, r, r, i, i, Y, ...",[0.11739850789308548],[0.13095000386238098],[0.05223875492811203],[157.65726394121165],[-22.807671218881783],[157.65725708007812],[-22.807472229003906],82113741,163
3,18,21885874059,121,"[60402.2838, 60373.1789, 60373.2036, 60374.358...","[2180.092041015625, 21.62727165222168, -357.32...","[384.5772705078125, 132.51869201660156, 165.11...","[z, g, r, i, i, u, u, g, g, u, u, g, u, r, u, ...",[0.039679743349552155],[0.0010000000474974513],[0.05719093605875969],[182.52191475033388],[-31.762338815591793],[182.52183532714844],[-31.76241111755371],10942937,156
4,25,22178354056,121,"[60505.9886, 60485.2627, 60485.2858, 60487.110...","[1518.2388916015625, -132.09169006347656, 3814...","[575.2952880859375, 1908.2532958984375, 1704.3...","[i, Y, Y, z, i, r, r, i, i, g, r, r, i, u, r, ...",[0.4057801365852356],[0.4137299954891205],[0.11362776160240173],[261.71389743791696],[-56.57013324218278],[261.7138671875],[-56.570159912109375],11089177,173


In [13]:
sub = pl.from_pandas(sub)

In [14]:
df = sub.explode(["cmidPointTai", "cpsFlux", "cpsFluxErr", "cfilterName"]).sort(by=["index", "cfilterName"])

df = df.explode(["cZ", "cZerr", "cMwebv", "cRa", "cDecl", "cHostgal_ra", "cHostgal_dec"])

df = df.rename({"SNID": "object_id","cmidPointTai": "mjd", "cpsFlux": "flux", "cpsFluxErr": "flux_error", "cfilterName": "filter", "cZ": "z", "cZerr": "z_error", "cMwebv": "mwebv", "cRa": "ra", "cDecl": "dec", "cHostgal_ra": "hostgal_ra", "cHostgal_dec": "hostgal_dec", "NOBS":"nobs"})

df

index,alertId,target,mjd,flux,flux_error,filter,z,z_error,mwebv,ra,dec,hostgal_ra,hostgal_dec,object_id,nobs
i64,i64,i64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,i64,i64
5,164227482030,121,60314.3082,-909.906738,875.011841,"""Y""",0.117399,0.13095,0.052239,157.657264,-22.807671,157.657257,-22.807472,82113741,163
5,164227482030,121,60327.3531,-1183.322876,872.459351,"""Y""",0.117399,0.13095,0.052239,157.657264,-22.807671,157.657257,-22.807472,82113741,163
5,164227482030,121,60327.3627,504.018555,972.680969,"""Y""",0.117399,0.13095,0.052239,157.657264,-22.807671,157.657257,-22.807472,82113741,163
5,164227482030,121,60335.3411,-1140.665161,1279.748901,"""Y""",0.117399,0.13095,0.052239,157.657264,-22.807671,157.657257,-22.807472,82113741,163
5,164227482030,121,60335.3513,340.209076,1157.631958,"""Y""",0.117399,0.13095,0.052239,157.657264,-22.807671,157.657257,-22.807472,82113741,163
5,164227482030,121,60339.3362,6172.255859,1754.083862,"""Y""",0.117399,0.13095,0.052239,157.657264,-22.807671,157.657257,-22.807472,82113741,163
5,164227482030,121,60314.2508,-231.218201,310.829102,"""i""",0.117399,0.13095,0.052239,157.657264,-22.807671,157.657257,-22.807472,82113741,163
5,164227482030,121,60318.2694,358.595886,264.157166,"""i""",0.117399,0.13095,0.052239,157.657264,-22.807671,157.657257,-22.807472,82113741,163
5,164227482030,121,60324.2566,147.180511,222.996368,"""i""",0.117399,0.13095,0.052239,157.657264,-22.807671,157.657257,-22.807472,82113741,163


In [15]:
len(df["alertId"].unique())

5

In [16]:
from astronet.preprocess import generate_gp_all_objects, remap_filters, generate_gp_single_event
from astronet.constants import *

In [17]:
df = remap_filters(df, filter_map=ELASTICC_FILTER_MAP)
df

ComputeError: ColumnNotFoundError: filter

> Error originated just after operation: '  LEFT JOIN:
  LEFT PLAN ON: [col("index")]
    DF ["index"]; PROJECT */1 COLUMNS; SELECTION: "None"
  RIGHT PLAN ON: [col("__POLARS_REMAP_KEY_index")]
     WITH_COLUMNS:
     [true.alias("__POLARS_REMAP_IS_REMAPPED_index")]
      DF ["__POLARS_REMAP_KEY_index", "__POLARS_REMAP_VALUE_index"]; PROJECT */2 COLUMNS; SELECTION: "None"
  END LEFT JOIN
'
This operation could not be added to the plan.

In [ ]:
df.shape[1]

In [ ]:
ddf = df.drop(columns=["index"])

In [ ]:
alert_list = list(np.unique(ddf["object_id"]))
len(alert_list)

In [ ]:
# %%timeit -n1 -r1
generated_gp_dataset = generate_gp_all_objects(alert_list, ddf)

In [ ]:
generated_gp_dataset

In [ ]:
ddf = df.drop(columns=["mjd", "filter", "flux", "flux_error"])

In [ ]:
ddf

In [ ]:
df_with_xfeats = generated_gp_dataset.merge(ddf, on="object_id", how="inner")
df_with_xfeats

In [ ]:
df_with_xfeats.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [ ]:
df_with_xfeats

In [ ]:
len(generated_gp_dataset["object_id"].unique())

In [ ]:
from astronet.viz.visualise_data import *
import random

In [ ]:
data = ddf[ddf["object_id"] == random.choice(alert_list)]

In [ ]:
data

In [ ]:
len(data["filter"].unique())

In [ ]:
_obj_gps = generate_gp_single_event(data)
ax = plot_event_data_with_model(data, obj_model=_obj_gps, pb_colors=ELASTICC_PB_COLORS)

In [ ]:
_obj_gps

In [ ]:
_obj_gps["filter"].unique()

In [ ]:
# N = 3
# seed = np.random.seed(1)
# indices = np.random.randint(0, len(sub), N)

# for index in indices:
#     fig = plt.figure(figsize=(12, 6))
    
#     data = sub.iloc[index]
#     filters = np.unique(data['cfilterName'])
    
#     for filt in filters:
#         cond = data['cfilterName'] == filt
        
#         plt.errorbar(
#             data['cmidPointTai'][cond], 
#             data['cpsFlux'][cond], 
#             data['cpsFluxErr'][cond],
#             ls='',
#             marker='o',
#             label='{} band'.format(filt)
#         )
#     plt.title('ID {} -- class: {}'.format(data['alertId'], data['classId']))
#     plt.legend()
#     plt.show()